In [ ]:
from core.manipulation_set import FrequencyManipulationSet, RGBManipulationSet

import hydra
from hydra import compose, initialize
import torch
import torch.multiprocessing

import matplotlib.pyplot as plt
import random

from core.utils import read_target_image
from experiments.eval_utils import feature_visualisation

torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=8)

In [ ]:
with initialize(version_base=None, config_path="../config"):
    cfg = compose(
        config_name="config_rs50_dalmatian_tunnel",
        overrides=[],
    )
torch.multiprocessing.set_sharing_strategy("file_system")

In [ ]:
class VisualizationUnit(torch.nn.Module):
    def __init__(
        self, norm_target, n_channels, image_dims, filter_dim=None, out_channels=1
    ):
        super(VisualizationUnit, self).__init__()

        filter_dim = filter_dim or image_dims
        self.out_channels = out_channels
        self.filter_dim = filter_dim
        self.image_dims = image_dims
        self.norm_target_shape = norm_target.shape

        # Define a single Conv2d layer (no training required)
        self.conv = torch.nn.Conv2d(
            in_channels=n_channels,
            out_channels=out_channels,  # Single output
            kernel_size=filter_dim,
            stride=(2, 2),
            padding=(3, 3),
            bias=False,
        )
        self.max_pool = torch.nn.MaxPool2d(
            kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False
        )
        self.avg_pool = torch.nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.relu = torch.nn.ReLU()

        self.rand_xs = [
            0,
            self.norm_target_shape[2] - self.filter_dim,
            self.norm_target_shape[2] - self.filter_dim,
            0,
        ]
        self.rand_ys = [
            0,
            self.norm_target_shape[3] - self.filter_dim,
            0,
            self.norm_target_shape[3] - self.filter_dim,
        ]

        # Set weights directly from norm_target
        with torch.no_grad():
            for n in range(self.out_channels):
                # find a subimage of the target images in the size of the kernel
                if n < 4:
                    rand_x = self.rand_xs[n]
                    rand_y = self.rand_ys[n]
                else:
                    rand_x = random.randint(
                        0, self.norm_target_shape[2] - self.filter_dim
                    )
                    rand_y = random.randint(
                        0, self.norm_target_shape[3] - self.filter_dim
                    )
                subimage = norm_target[
                    :,
                    :,
                    rand_x : rand_x + self.filter_dim,
                    rand_y : rand_y + self.filter_dim,
                ]
                self.conv.weight[n].copy_(subimage[0])

            self.max = self.avg_pool(self.max_pool(self.conv(norm_target)))

    def forward(self, x):
        # Apply the convolution
        return -self.relu(-self.avg_pool(self.max_pool(self.conv(x))) + self.max)

In [ ]:
device = cfg.device
dataset = cfg.data
image_dims = int(cfg.data.image_dims)
n_channels = int(cfg.data.n_channels)
fv_sd = float(cfg.fv_sd)
fv_dist = cfg.fv_dist
fv_domain = cfg.fv_domain
target_img_path = "." + cfg.target_img_path
zero_rate = cfg.get("zero_rate", 0.5)
tunnel = cfg.get("tunnel", False)
target_noise = float(cfg.get("target_noise", 0.0))

transforms = hydra.utils.instantiate(dataset.fv_transforms)
normalize = hydra.utils.instantiate(cfg.data.normalize)
denormalize = hydra.utils.instantiate(cfg.data.denormalize)
resize_transforms = hydra.utils.instantiate(cfg.data.resize_transforms)

if "target_act_fn" in cfg.model:
    if "probe_path" in cfg.model.target_act_fn:
        cfg.model.target_act_fn.probe_path = "." + cfg.model.target_act_fn.probe_path
    target_act_fn = hydra.utils.instantiate(cfg.model.target_act_fn)
else:
    target_act_fn = lambda x: x

default_model = hydra.utils.instantiate(cfg.model.model)
default_model.to(device)

In [ ]:
noise_ds_type = FrequencyManipulationSet if fv_domain == "freq" else RGBManipulationSet
noise_dataset = noise_ds_type(
    image_dims,
    target_img_path,
    normalize,
    denormalize,
    transforms,
    resize_transforms,
    n_channels,
    fv_sd,
    fv_dist,
    zero_rate,
    tunnel,
    device,
)

In [ ]:
norm_target, _ = read_target_image(device, n_channels, target_img_path, normalize)
# create conv layer encoding norm_target
model = VisualizationUnit(norm_target.to("cpu"), n_channels, image_dims, 64, 512)

model.to(device)
model.conv.to(device)
model.avg_pool = model.avg_pool.to(device)
model.max_pool = model.max_pool.to(device)
model.max = model.max.to(device)
model.eval()

In [ ]:
img, _, tstart = feature_visualisation(
    net=model,
    noise_dataset=noise_dataset,
    man_index=0,
    lr=0.001,
    n_steps=100,
    target_act_fn=target_act_fn,
    grad_clip=True,
    adam=True,
    device=device,
)
plt.imshow(img[0].permute(1, 2, 0).detach().cpu().numpy())
plt.show()

In [ ]:
kernel_size = 164
conv = torch.nn.Conv2d(
    in_channels=n_channels,
    out_channels=2,  # Single output
    kernel_size=kernel_size,
    # Kernel size matches the input spatial dimensions
    stride=1,
    padding=0,
    bias=False,  # No bias for simplicity
)
for n in range(2):
    # find a subimage of the target images in the size of the kernel
    rand_x = random.randint(0, norm_target.shape[2] - kernel_size)
    rand_y = random.randint(0, norm_target.shape[3] - kernel_size)
    subimage = norm_target[
        :, :, rand_x : rand_x + kernel_size, rand_y : rand_y + kernel_size
    ]
    conv.weight.data[n] = subimage

conv(norm_target.to("cpu")).shape

In [ ]:
import torchvision.models as models

resnet18 = models.resnet18(pretrained=True)
resnet18.eval()